In [1]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import folium
from folium.plugins import HeatMap

Cada carpeta de dentro de la carpeta `data` contiene dos ficheros `.csv`:
- Uno para los alquileres diarios de cada ciudad
- Información de cada apartamento

---

# 1. Datos de los apartamentos

In [ ]:
data_path = os.path.join(os.getcwd(), 'data')

guardamar_rents = pd.read_csv(os.path.join(data_path, 'Guardamar', 'Spain-guardamar-del-segura_Daily_Match_2023-11-23.csv'))
guardamar_info = pd.read_csv(os.path.join(data_path, 'Guardamar', 'Spain-guardamar-del-segura_Property_Extended_Match_2023-11-23.csv'))

In [ ]:
def get_tipos_de_propiedad(apartments_info_df):
	bar_fig = px.bar(
		apartments_info_df['Property Type'].value_counts().reset_index(),
		x='count',
		y='Property Type',
		title='Tipos de propiedades en Guardamar del Segura',
		labels={'count': 'Cantidad', 'Property Type': 'Tipo de propiedad'},
		color='Property Type',
		color_discrete_sequence=px.colors.qualitative.Plotly,
	)

	bar_fig.update_traces(marker=dict(line=dict(width=1, color='black')))

	bar_fig.update_layout(
		title_font=dict(size=20),
		title_x=0.5,
		title_y=0.95,
		xaxis_title='Cantidad',
		yaxis_title='Tipo de propiedad',
		xaxis=dict(showgrid=False, zeroline=False),
		yaxis=dict(showgrid=False, zeroline=False),
		showlegend=False,
	)

	bar_fig.show()

def get_avg_price_per_type(apartments_info_df):
	avg_rent_per_property = apartments_info_df[apartments_info_df['Average Daily Rate (Native)'] > 0].groupby("Property Type")["Average Daily Rate (Native)"].mean().reset_index()
	avg_rent_per_property = avg_rent_per_property.rename(columns={"Average Daily Rate (Native)": "Average Daily Rate"})

	fig = px.bar(
			avg_rent_per_property,
			x="Property Type",
			y="Average Daily Rate",
			color="Property Type",
			title="Average Daily Rate by Property Type in Guardamar del Segura",
			color_discrete_sequence=px.colors.qualitative.Plotly,
			labels={"Property Type": "Tipo de propiedad", "Average Daily Rate": "Precio medio diario (€)"},
		)

	fig.show()

In [ ]:
def get_rel_btw_rooms(apartments_info_df):
	fig = px.scatter(
		apartments_info_df,
		x='Bedrooms',
		y='Max Guests',
		size='Bathrooms',
		color='Property Type',
		title='Relación entre habitaciones, capacidad y baños',
		hover_data=['Average Daily Rate (Native)'],
		labels={
			'Bedrooms': 'Habitaciones',
			'Max Guests': 'Capacidad',
			'Bathrooms': 'Baños',
			'Average Daily Rate (Native)': 'Precio medio diario (€)',
			'Property Type': 'Tipo de propiedad'
		},
	)

	fig.update_layout(title_x=0.5)

	fig.show()

	fig = px.box(
		apartments_info_df,
		x='Bedrooms',
		y='Max Guests',
		color='Bedrooms',
		title='Capacidad máxima por número de habitaciones',
		points='all',
		labels={
			'Bedrooms': 'Habitaciones',
			'Max Guests': 'Capacidad',
			'Property Type': 'Tipo de propiedad'
		},
		hover_data=['Average Daily Rate (Native)'],
	)

	fig.update_layout(title_x=0.5, showlegend=False)
	fig.show()



get_rel_btw_rooms(guardamar_info)

In [ ]:
def get_acomm_distribution(apartments_info_df, accommodates):
	fig = px.histogram(
		apartments_info_df,
		x=accommodates,
		title=f'Distribución de {accommodates} en Guardamar del Segura',
		color_discrete_sequence=px.colors.qualitative.Plotly,
		marginal='box',
		hover_data=apartments_info_df.columns
	)

	fig.update_layout(
		title_font=dict(size=20),
		title_x=0.5,
		xaxis_title=f'Número de {accommodates}',
		yaxis_title='Cantidad de propiedades',
		xaxis=dict(showgrid=False, zeroline=False),
		yaxis=dict(showgrid=False, zeroline=False),
		showlegend=False
	)

	fig.show()

get_acomm_distribution(guardamar_info, 'Max Guests')

In [ ]:
def get_price_vs_capacity(apartments_info_df):
	fig = px.scatter(
		apartments_info_df,
		x='Max Guests',
		y='Average Daily Rate (Native)',
		color='Property Type',
		title='Precio medio diario vs Capacidad máxima',
		labels={
			'Max Guests': 'Capacidad máxima',
			'Average Daily Rate (Native)': 'Precio medio diario (€)',
			'Property Type': 'Tipo de propiedad'
		},
	)

	fig.update_layout(title_x=0.5)

	fig.show()

get_price_vs_capacity(guardamar_info)

In [ ]:
map = folium.Map(location=[guardamar_info['Latitude'].mean(), guardamar_info['Longitude'].mean()], zoom_start=13)

heat_data = [[row['Latitude'], row['Longitude'], row['Average Daily Rate (Native)']] for index, row in guardamar_info.iterrows()]
heat_data = [[row['Latitude'], row['Longitude'], row['Average Daily Rate (Native)']] for index, row in guardamar_info.iterrows() if row['Average Daily Rate (Native)'] > 0]

heatmap = HeatMap(heat_data, radius=15, blur=10, max_zoom=1)
map.add_child(heatmap)
map.add_child(folium.LayerControl())

# map

---

# 2. Datos de alquileres

Ideas:
- ¿Cuántos días se alquila cada apartamento?
- ¿Cuántos días se alquila cada apartamento por ciudad?
- ¿Cuántos días se alquila cada apartamento por ciudad y por año?
- Análisis de ocupación media por mes o semana
- Cambios de precio a lo largo del año
- Propiedades con más ingresos totales = precio diario × días ocupados
- Propiedades que ajustan más sus precios (volatilidad)
- ¿Qué tipo de alojamiento es más rentable?
- ¿Cuándo es más caro alojarse en Guardamar del Segura?
- ¿Cómo afecta la temporada alta a la disponibilidad y precios?
- ¿Qué zonas tienen los alojamientos más caros?
- ¿Hay propietarios que tienen múltiples alojamientos (multi-host)?

- Clusterización de propiedades por precio, tamaño, tipo → segmentación del mercado
- Predicción de precios con modelos simples tipo regresión

In [ ]:
# common_cols = list(set(guardamar_rents.columns) & set(guardamar_info.columns))
# common_cols = [col for col in common_cols if col != 'Property ID']

# guardamar_info_clean = guardamar_info.drop(columns=common_cols)

# merged = guardamar_rents.merge(guardamar_info_clean, how='inner', on='Property ID')

In [ ]:
## Calendario de precios (available, price) a lo largo del tiempo
def get_price_calendar(merged_df):
	merged_df['Date'] = pd.to_datetime(merged_df['Date'])
	merged_df['Average Daily Rate (Native)'] = merged_df['Average Daily Rate (Native)'].astype(float)

	fig = px.line(
		merged_df,
		x='Date',
		y='Average Daily Rate (Native)',
		color='Property ID',
		title='Calendario de precios en Guardamar del Segura',
		labels={'Date': 'Fecha', 'Average Daily Rate (Native)': 'Precio (€)', 'Property ID': 'ID de propiedad'},
	)

	fig.update_layout(title_x=0.5, title_y=0.95)
	fig.show()

get_price_calendar(merged)
